In [367]:
# Import potrebných knihoven
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, mannwhitneyu, shapiro

# Načtení datasetu
df_accidents = pd.read_pickle("accidents.pkl.gz")
df_vehicles = pd.read_pickle("vehicles.pkl.gz")

# Test hypotézy (xhejni00)

## 1. příklad

### Zadaní (hypotéza 1): Na silnicích první třídy byly nehody s následky na zdraví se stejnou pravděpodobností jako na dálnicích.

Hypotéza 1 je zároveň hypotézou nula pro náš chí kvadrát test. H0: Na silnicích I. třídy byly dané nehody se stejnou pravděpodobností jako na dálnicích
Hypotéza H1: Na silnicích I. třídy nejsou dané nehody se stejnou pravděpodobností jako na dálnících.

Daným nehodám rozumějme nehody s následky na zdraví.

Nejprve vyfiltrujeme pouze nehody, které se staly na dálnici nebo silnici první třídy

In [368]:
df_accidents = df_accidents[df_accidents['p36'].isin([0, 1])]

Dále vytvoříme dva binární sloupce "highway" a "injury". Hodnoty ve sloupci "highway" nabývají hodnoty True pokud se jedná o dálnici, False v opačném případě. Hodnoty v druhém slpupci nabývají hodnoty True pokud došlo k nehodě s následky na zdraví, False v opačném případě

In [369]:
df_accidents['highway'] = df_accidents['p36'] == 0 
df_accidents['injury'] = df_accidents['p9'] == 1

Dalším krokem je vytvoření kontigenční tabulky pro dané sloupce. Ta nám pomůže vizualizovat vztah dvou kategorií statistických znaků. V našem případě se jedná o vztah mezi typem silnice a nehodami s následky na zdraví.

In [370]:
ct = pd.crosstab(df_accidents['highway'], df_accidents['injury'])
ct

injury,False,True
highway,,
False,14773,7059
True,6674,1247


Tuto kontingenční tabulku využijeme pro chí kvadrát test, který je spočítán funkcí **chi2_contingency**. **P-value** získanou tímto testem použijeme pro prokázání nebo zamítnutí nulové hypotézy. Zadání požaduje hladinu významnosti **95 %**, tedy alfa je 0.05. Proti této hodně dále postavíme získanou p-hodnotu.

In [371]:
result = chi2_contingency(ct)
print(f'P-hodnota: {result.pvalue}')

if result.pvalue < 0.05:
    print("Zamítáme nulovou hypotézu, tedy jsme nepotvrdili stejnou míru zranění na dálnici a na silnicích první třídy.")
else:
    print("Nezamitáme nulovou hypotézu, a tedy je pravděpodobnost daných nehod s následky na zdraví na obou typech silnic stejná.")

P-hodnota: 1.0075002730159729e-174
Zamítáme nulovou hypotézu, tedy jsme nepotvrdili stejnou míru zranění na dálnici a na silnicích první třídy.


Vzhledem k výsledku p-value, která je velice malá (asi 1.0e-174) **zamítáme nulovou hypotézu**, tedy říkáme že s 95 % jistotou nejsou na obou typech silnic nehody s následky na zdraví se stejnou pravděpodobností.

V neposlední řadě zodpovíme otázku, zdali nehody na silnicích 1. třídy vedly méně nebo více často k nehodě s následky na zdraví. K tomu využijeme výstup expected, který vrací již dříve použitá funkce pro výpočet testu. 

In [372]:
print(result.expected_freq)

[[15737.26696468  6094.73303532]
 [ 5709.73303532  2211.26696468]]


Tato tabulka nám říká očekávané hodnoty v kontingenční tabulce, aby nedošlo k zamítnutí nulové hypotézy. **V našem i tomto případě lze říci, že na silnicích I. třídy dochází k nehodám s následky na zdraví častěji než na dálnicích.**

## 2. příklad

### Zadaní (hypotéza 2): Škoda při nehodách trolejbusů je nižší, než při nehodách autobusů a tato odchylka je statisticky významná.

Jako nulovou hypotézu H0 zvolíme, že škoda při nehodách trolejbusů **není nižší** než při nehodách autobusů. (v dokumentaci [funkce](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html) použité pro test, kde srovnávali muže a ženy měli nulovou hypotézu vyjádřenou stejně)
 
Jako alternativní hypotézu H1 zvolíme tu ze zadaní, tedy že škoda při nehodách trolejbusů **je nižší** než při nehodách autobusů (ta je parametrem pro test níže).

(V praxi by neměl být problém prohození obou hypotéz, jen je s nimi potřeba dobře dále pracovat.)

Nejprve je třeba nutné zjistit, zdali data pocházejí z **normálního rozdělení**, což zjistíme například pomocí **Shapiro-Wilk testu**. Díky zjištění rozdělení dat, dále můžeme určit vhodný statistický test.

Před provedením testu jsou vytvořeny dva datasety pocházející z původního datasetu - jeden pro autobusy a druhý pro trolejbusy. Pro sloupec "p53", který reprezentuje škodu na vozidle ve stokorunách, je proveden Shapiro-Wilk test ověřující normalitu dat.

In [373]:
df_bus = df_vehicles[df_vehicles['p44'] == 8]
df_trolleybus = df_vehicles[df_vehicles['p44'] == 11]

busNormality = shapiro(df_bus['p53'])
trolleybusNormality = shapiro(df_trolleybus['p53'])

print(f'Výsledná p-hodnota pro autobusy: {busNormality.pvalue}')
print(f'Výsledná p-hodnota pro trolejbusy: {trolleybusNormality.pvalue}')

if busNormality.pvalue < 0.05:
    print("Data škod na autobusech nemají normální rozdělení.")
else:
    print("Data škod na autobusech mají normální rozdělení.")

if trolleybusNormality.pvalue < 0.05:
    print("Data škod na trolejbusech nemají normální rozdělení.")
else:
    print("Data škod na trolejbusech mají normální rozdělení.")


Výsledná p-hodnota pro autobusy: 5.286327503539159e-82
Výsledná p-hodnota pro trolejbusy: 3.0873452435991516e-31
Data škod na autobusech nemají normální rozdělení.
Data škod na trolejbusech nemají normální rozdělení.


Výsledek ukazuje, že data obou dopravních prostředků **nemají normální rozdělení** a nemůžeme tedy počítat s průměry, jelikož hodnota p-value, která je výsledkem tohoto testu je menší než 0.05, což je hladina významnosti, zvolená zadáním. V takovém případě použijeme Mann-Whitneyho U-Test, který je neparametrickou verzí t-testu pro nezávislé vzorky. Jedná se o test nulové hypotézy, že pro náhodně vybrané hodnoty X a Y ze dvou populací (v našem případě typů vozidel) se pravděpodobnost, že X bude větší než Y, rovná pravděpodobnosti, že Y
bude větší než X (tedy že hodnoty jsou přibližně stejné).

Ještě před samotným testem je dobré se podívat na mediány škod na daných vozidlech, které jsou převedeny na Kč. Ty nám ukazují, že škoda na autobusech je násobně větší než na trolejbusech.

In [374]:
print(f'Medián škod autobusu: {(df_bus["p53"].median() * 100):.0f} Kč')
print(f'Medián škod trolejbusu: {(df_trolleybus["p53"].median() * 100):.0f} Kč')

Medián škod autobusu: 20000 Kč
Medián škod trolejbusu: 5000 Kč


Vstupem do Mann-Whitneyho testu jsou jednotlivé hodnoty škod na dle typu vozidla a hodnota parametru **alternative** "less", která specifikuje jednostranný test (testuje zda hodnoty z 1. skupiny jsou menší než ze 2. skupiny). To odpovídá alternativní hypotéze H1, tedy že škoda při nehodách trolejbusu je nižší než při nehodách autobusu. Výsledkem je jako v případě 1. příkladu p-hodnota, která určí zdali zamítáme nulovou hypotézu či nikoliv.

In [375]:
result = mannwhitneyu(df_trolleybus['p53'], df_bus['p53'], alternative='less')
print(f'Výsledná p-hodnota: {result.pvalue}')

if result.pvalue < 0.05:
    print("Zamítáme hypotézu H0 ve prospěch alternativní hypotézy, tedy s 95 % spolehlivostí říkáme, že škody při nehodě trolejbusu jsou statisticky nižší oproti autobusu.")
else:
    print("Nezamítáme nulovou hypotézu, a tedy jsme neprokázali statisticky nižší škodu při nehodách trolejbusu oproti autobusu (nemáme dost důkazů).")

Výsledná p-hodnota: 1.7702098289956038e-13
Zamítáme hypotézu H0 ve prospěch alternativní hypotézy, tedy s 95 % spolehlivostí říkáme, že škody při nehodě trolejbusu jsou statisticky nižší oproti autobusu.


Výsledek ukazuje, že s 95 % spolehlivostí **zamítáme naši nulovou hypotézu**, a tedy se potvrdilo, že trolejbusy mají **nižší** škody než autobusy. Tímto testem jsme tedy potvrdili prvotní odhad z mediánů škod na vozidlech a zároveň statistickou významnost tohoto rozdílu.